In [3]:
# Import necessary libraries
import requests
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer, PorterStemmer
from textblob import TextBlob
import sys

# Load data from an Excel file
df = pd.read_excel(r'C:\Users\Desktop\RakBank Facebook.xlsx')

# Rename columns for clarity
df = df.rename(columns={'comment_count': 'Comments_Count', 'like_count': 'Like_Count'})
df['query_type'].replace('Facebook:/<page-id>/posts', 'Posts', inplace=True)
df['query_type'].replace('Facebook:/<post-id>/comments', 'Comments', inplace=True)

# Shift values in the 'Like_Count' and 'Comments_Count' columns
df['Like_Count'] = df.Like_Count.shift(2)
df['Comments_Count'] = df.Comments_Count.shift(2)

# Drop rows where 'query_type' is 'Facebook:/<post-id>'
df.drop(df.loc[df['query_type'] == 'Facebook:/<post-id>'].index, inplace=True)

# Define a function for sentiment analysis
def sentiscore(x):
    return TextBlob(x).sentiment[0]

def sentimag(x):  
    return TextBlob(x).sentiment[1]

def sentinew(x):  
    return TextBlob(x).sentiment

# Change 'message' column to string type
df['message'] = df['message'].astype(str)

# Convert messages to lowercase
messages = df['message'].apply(lambda x: " ".join(x.lower() for x in x.split()))

# Remove punctuation from messages
messages = messages.str.replace('[^\w\s]', '')

# Stemming
st = PorterStemmer()
messages = messages.apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

# Create a DataFrame for sentiment analysis
sentiment_dataframe = pd.DataFrame()

# Calculate sentiment scores and add them to the DataFrame
sentiment_dataframe['messagesscore'] = messages.apply(sentiscore)
sentiment_dataframe['messagesmagnitude'] = messages.apply(sentimag)
sentiment_dataframe['subjectivity/polarity'] = messages.apply(sentinew)

# Create DataFrames for other relevant columns
messages = pd.DataFrame(messages)
parent_id = pd.DataFrame(df['parent_id'])
query_type = pd.DataFrame(df['query_type'])
date = pd.DataFrame(df['created_date'])
time = pd.DataFrame(df['created_time'])
likes = pd.DataFrame(df['Like_Count'])
comments = pd.DataFrame(df['Comments_Count'])

# Combine all DataFrames
sentiment_dataframe = sentiment_dataframe.join(messages)
sentiment_dataframe = sentiment_dataframe.join(query_type)
sentiment_dataframe = sentiment_dataframe.join(parent_id)
sentiment_dataframe = sentiment_dataframe.join(date)
sentiment_dataframe = sentiment_dataframe.join(time)
sentiment_dataframe = sentiment_dataframe.join(likes)
sentiment_dataframe = sentiment_dataframe.join(comments)

# Convert sentiment scores to float
sentiment_dataframe['messagesscore'] = sentiment_dataframe['messagesscore'].apply(lambda x: float(x))

# Create a DataFrame for 'Sentiments'
Sentiment = pd.DataFrame(columns=['Sentiments'])

# Classify sentiments based on sentiment scores
sentiment_dataframe['Sentiments'] = np.where((sentiment_dataframe['messagesscore'] > 0).any() and
                                             (sentiment_dataframe['messagesscore'] <= 0.5).any(), 'Good',
                                             sentiment_dataframe['Sentiments'])
sentiment_dataframe['Sentiments'] = np.where(sentiment_dataframe['messagesscore'] == 0, 'Neutral',
                                             sentiment_dataframe['Sentiments'])
sentiment_dataframe['Sentiments'] = np.where(sentiment_dataframe['messagesscore'] > 0.5, 'Very Good',
                                             sentiment_dataframe['Sentiments'])
sentiment_dataframe['Sentiments'] = np.where(sentiment_dataframe['messagesscore'] < 0, 'Bad',
                                             sentiment_dataframe['Sentiments'])
sentiment_dataframe['Sentiments'] = np.where(sentiment_dataframe['messagesscore'] < -0.5, 'Very Bad',
                                             sentiment_dataframe['Sentiments'])

# Drop the first row and save the resulting DataFrame to a CSV file
sentiment_dataframe = sentiment_dataframe.iloc[1:, :]
sentiment_dataframe.to_csv(r'C:\Users\RakBankFacebookSentiments.csv', index=None, header=True)


<ipython-input-3-37f5edd6b570>:62: FutureWarning: The default value of regex will change from True to False in a future version.
  messages  = messages.str.replace('[^\w\s]','')
